In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Define file paths for AQI and pollutant data
years = [2023,2022]
aqi_files = {
    2023:"/Users/shishir/Downloads/cs667-project/AQI DATA/AQI_daily_city_level_kanpur_2023_kanpur_2023.xlsx",
    2022: "/Users/shishir/Downloads/cs667-project/AQI DATA/AQI_daily_city_level_kanpur_2022_kanpur_2022.xlsx",
}
pollutant_files = {
    2023:"/Users/shishir/Downloads/cs667-project/POLLUTANT DATA/Raw_data_1Day_2023_site_5500_FTI_Kidwai_Nagar_Kanpur_UPPCB_1Day.csv",
    2022: "/Users/shishir/Downloads/cs667-project/POLLUTANT DATA/Raw_data_1Day_2022_site_5500_FTI_Kidwai_Nagar_Kanpur_UPPCB_1Day.csv",
  
}
# Prepare an empty list to store merged data for each year
merged_yearly_data = []

# Month mapping for reshaping AQI data
month_mapping = {
    "January": 1, "February": 2, "March": 3, "April": 4, "May": 5, "June": 6, 
    "July": 7, "August": 8, "September": 9, "October": 10, "November": 11, "December": 12
}

# Loop through each year to load, process, and merge data
for year in years:
    # Load AQI and pollutant data for the year
    aqi_data = pd.read_excel(aqi_files[year])
    pollutant_data = pd.read_csv(pollutant_files[year])
    
    # Convert the timestamp in pollutant data to datetime and set as index
    pollutant_data['Timestamp'] = pd.to_datetime(pollutant_data['Timestamp'])
    pollutant_data.set_index('Timestamp', inplace=True)
    pollutant_data_interpolated = pollutant_data.interpolate(method='time')
    # Resample pollutant data to daily averages
    daily_pollutant_data = pollutant_data.resample('D').mean().reset_index()
    
    # Reshape AQI data to long format
    aqi_data = aqi_data.rename(columns={"Date": "Day"})
    aqi_long = pd.melt(aqi_data, id_vars=['Day'], var_name='Month', value_name='AQI')
    aqi_long['Month'] = aqi_long['Month'].map(month_mapping)
    aqi_long['Year'] = year
    # Clean the data to remove any non-numeric values
    aqi_long = aqi_long.dropna(subset=['Day', 'Month', 'Year'])  # Drop rows with missing values in date columns
    aqi_long['Day'] = pd.to_numeric(aqi_long['Day'], errors='coerce')
    aqi_long['Month'] = pd.to_numeric(aqi_long['Month'], errors='coerce')
    aqi_long['Year'] = pd.to_numeric(aqi_long['Year'], errors='coerce')
    aqi_long = aqi_long.dropna(subset=['Day', 'Month', 'Year'])  # Drop rows where conversion failed

    # Convert to datetime
    aqi_long['Date'] = pd.to_datetime(aqi_long[['Year', 'Month', 'Day']], errors='coerce')
    aqi_long = aqi_long.dropna(subset=['Date'])  # Drop rows where date conversion failed
    aqi_long.drop(['Year', 'Month', 'Day'], axis=1, inplace=True)

    # Merge daily pollutant data with AQI data
    merged_data = pd.merge(daily_pollutant_data, aqi_long, left_on='Timestamp', right_on='Date')
    merged_data.drop(columns=['Date'], inplace=True)
    # Append merged data to the list
    merged_yearly_data.append(merged_data)

# Concatenate all years into a single DataFrame
combined_data = pd.concat(merged_yearly_data, ignore_index=True)
combined_data=combined_data.fillna(0, inplace=False)

# Display the final merged data structure
# combined_data.to_csv('/Users/shishir/Downloads/cs667-project/combined_data.csv', index=False)
# print(combined_data.head())
columns_to_keep = ['Timestamp', 'PM2.5 (µg/m³)', 'PM10 (µg/m³)','NH3 (µg/m³)','Benzene (µg/m³)','AT (°C)','RH (%)' , 'AQI']

filtered_data = combined_data[columns_to_keep]
print(filtered_data.head())
filtered_data.to_csv('/Users/shishir/Downloads/cs667-project/filtered_data.csv', index=False)

# Perform time-based interpolation to fill NaN values



   Timestamp  PM2.5 (µg/m³)  PM10 (µg/m³)  NH3 (µg/m³)  Benzene (µg/m³)  \
0 2023-05-01      19.369545     36.501932    27.693229         0.108750   
1 2023-05-02      20.539348     31.297283    26.885208         0.109271   
2 2023-05-03      20.206250     37.149091    25.037813         0.113958   
3 2023-05-04      18.622738     32.812143    26.226667         0.118750   
4 2023-05-05      31.897604     63.149167    25.659271         0.164062   

     AT (°C)     RH (%)   AQI  
0  26.177604  71.717292  53.0  
1  26.393750  71.775833  47.0  
2  26.516563  73.460938  52.0  
3  26.869167  72.293750  48.0  
4  29.086042  59.965313  52.0  


In [34]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split


In [35]:

data = pd.read_csv('/Users/shishir/Downloads/cs667-project/filtered_data.csv')
timestamps = data['Timestamp'].reset_index(drop=True)

features = ['PM2.5 (µg/m³)', 'PM10 (µg/m³)','NH3 (µg/m³)','Benzene (µg/m³)','AT (°C)','RH (%)' , 'AQI']
data = data[features]
print(data.head())

   PM2.5 (µg/m³)  PM10 (µg/m³)  NH3 (µg/m³)  Benzene (µg/m³)    AT (°C)  \
0      19.369545     36.501932    27.693229         0.108750  26.177604   
1      20.539348     31.297283    26.885208         0.109271  26.393750   
2      20.206250     37.149091    25.037813         0.113958  26.516563   
3      18.622738     32.812143    26.226667         0.118750  26.869167   
4      31.897604     63.149167    25.659271         0.164062  29.086042   

      RH (%)   AQI  
0  71.717292  53.0  
1  71.775833  47.0  
2  73.460938  52.0  
3  72.293750  48.0  
4  59.965313  52.0  


In [36]:
lagged_data = pd.read_csv('/Users/shishir/Downloads/cs667-project/lagged_data.csv')
lagged_data = lagged_data.drop(columns=['Timestamp'])

In [37]:
X = lagged_data.drop(columns=['AQI'])  
y = lagged_data['AQI'] 


In [38]:

split_ratio = 0.8
split_index = int(len(X) * split_ratio)
X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]


In [39]:
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)


RandomForestRegressor(random_state=42)

In [40]:
predictions = rf_model.predict(X_test)
rmse = mean_squared_error(y_test, predictions, squared=False)
print(f"RMSE: {rmse}")


RMSE: 38.225615190166224


/opt/anaconda3/envs/my_new_env/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [41]:
import joblib
joblib.dump(rf_model, '/Users/shishir/Downloads/cs667-project/rf_model.pkl')

['/Users/shishir/Downloads/cs667-project/rf_model.pkl']